In [ ]:
# %pip install transformers datasets

In [1]:
import numpy as np
import torch
from datasets import list_datasets, load_dataset
from transformers import AutoTokenizer, AutoModel, AutoModelForSequenceClassification, Trainer, TrainingArguments, pipeline
from sklearn.metrics import accuracy_score, f1_score

In [2]:
# DATA
# dataset = load_dataset('emotion')
dataset = load_dataset('tamilmixsentiment')
dataset


# PARAMS
num_labels = len(set(dataset['train']['label']))
print('num_labels:', num_labels)
model_ckpt = 'distilbert-base-uncased'


# DEVICE
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device


# TOKENS
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)


def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)


dataset_encoded = dataset.map(tokenize, batched=True, batch_size=None)
dataset_encoded


print(dataset_encoded['train'].format)
dataset_encoded.set_format('torch')
print(dataset_encoded['train'].format)


# MODEL
model = AutoModelForSequenceClassification.from_pretrained(
    model_ckpt, num_labels=num_labels
).to(device)

print('model.device', model.device)


def compute_metrics(pred):
    res = {'acc': 1.0, 'F1': 2.0}
    return res


batch_size = 64
training_args = TrainingArguments(
    output_dir='resultados',
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    disable_tqdm=False,
    evaluation_strategy='steps',
    logging_steps=batch_size,
    report_to='none',
)

trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=dataset_encoded['train'],
    eval_dataset=dataset_encoded['validation'],
    tokenizer=tokenizer,
)

print('trainer.args.device:', trainer.args.device)

Generating train split:   0%|          | 0/11335 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1260 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3149 [00:00<?, ? examples/s]

Dataset tamilmixsentiment downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/tamilmixsentiment/default/0.0.0/887420eecaf868ac6c10990649e49d10467e4cd4dffb98a6f20e4fe7c58df390. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

{'type': None, 'format_kwargs': {}, 'columns': ['text', 'label', 'input_ids', 'attention_mask'], 'output_all_columns': False}
{'type': 'torch', 'format_kwargs': {}, 'columns': ['text', 'label', 'input_ids', 'attention_mask'], 'output_all_columns': False}


Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.w

model.device cuda:0
trainer.args.device: cuda:0


In [3]:
trainer.train()
trainer.save_model('resultados2')

The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/home/ec2-user/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 11335
  Num Epochs = 2
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 356
  Number of trainable parameters = 66957317
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using th

Step,Training Loss,Validation Loss,Acc,F1
64,1.102400,0.968699,1.000000,2.000000
128,0.969000,0.921387,1.000000,2.000000
192,0.902100,0.913229,1.000000,2.000000
256,0.880300,0.891879,1.000000,2.000000
320,0.901300,0.884532,1.000000,2.000000


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1260
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1260
  Batch size = 64
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 

In [4]:
preds = trainer.predict(dataset_encoded['validation'])

The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1260
  Batch size = 64


In [5]:
preds

PredictionOutput(predictions=array([[ 2.4332113 , -0.31521487,  0.14545262, -0.82355124, -2.75796   ],
       [ 0.99884474,  0.8013246 ,  0.2724206 , -0.6925419 , -2.187095  ],
       [ 2.058402  ,  0.17668481,  0.32721087, -0.9802026 , -2.8120365 ],
       ...,
       [ 2.684277  , -0.4904425 ,  0.07915889, -1.0423839 , -2.6275113 ],
       [ 2.2797112 , -0.24299574,  0.12822396, -0.6842281 , -2.7492263 ],
       [ 2.7594373 , -0.45782337,  0.19309384, -1.0889416 , -2.7005286 ]],
      dtype=float32), label_ids=array([0, 1, 2, ..., 1, 3, 0]), metrics={'test_loss': 0.883865237236023, 'test_acc': 1.0, 'test_F1': 2.0, 'test_runtime': 1.6105, 'test_samples_per_second': 782.387, 'test_steps_per_second': 12.419})

In [6]:
preds.metrics

{'test_loss': 0.883865237236023,
 'test_acc': 1.0,
 'test_F1': 2.0,
 'test_runtime': 1.6105,
 'test_samples_per_second': 782.387,
 'test_steps_per_second': 12.419}

In [7]:
preds.predictions

array([[ 2.4332113 , -0.31521487,  0.14545262, -0.82355124, -2.75796   ],
       [ 0.99884474,  0.8013246 ,  0.2724206 , -0.6925419 , -2.187095  ],
       [ 2.058402  ,  0.17668481,  0.32721087, -0.9802026 , -2.8120365 ],
       ...,
       [ 2.684277  , -0.4904425 ,  0.07915889, -1.0423839 , -2.6275113 ],
       [ 2.2797112 , -0.24299574,  0.12822396, -0.6842281 , -2.7492263 ],
       [ 2.7594373 , -0.45782337,  0.19309384, -1.0889416 , -2.7005286 ]],
      dtype=float32)

In [8]:
np.argmax(preds.predictions, axis=1)

array([0, 0, 0, ..., 0, 0, 0])

In [9]:
pipe = pipeline("text-classification", 'resultados2')

loading configuration file resultados2/config.json
Model config DistilBertConfig {
  "_name_or_path": "resultados2",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.26.0",
  "vocab_size": 30522
}

loading configuration file resultados2/config.json
Model config DistilBertConfig {
  

In [10]:
res = pipe(dataset['validation']['text'])

Disabling tokenizer parallelism, we're using DataLoader multithreading already


In [12]:
res

[{'label': 'LABEL_0', 'score': 0.8267189264297485},
 {'label': 'LABEL_0', 'score': 0.3952564299106598},
 {'label': 'LABEL_0', 'score': 0.7220352292060852},
 {'label': 'LABEL_0', 'score': 0.41130590438842773},
 {'label': 'LABEL_0', 'score': 0.8072349429130554},
 {'label': 'LABEL_0', 'score': 0.8932523727416992},
 {'label': 'LABEL_0', 'score': 0.6636306047439575},
 {'label': 'LABEL_0', 'score': 0.8774976134300232},
 {'label': 'LABEL_0', 'score': 0.7563446760177612},
 {'label': 'LABEL_0', 'score': 0.8971289396286011},
 {'label': 'LABEL_0', 'score': 0.5913744568824768},
 {'label': 'LABEL_0', 'score': 0.6116124987602234},
 {'label': 'LABEL_0', 'score': 0.6415722370147705},
 {'label': 'LABEL_0', 'score': 0.6657838821411133},
 {'label': 'LABEL_0', 'score': 0.8902761340141296},
 {'label': 'LABEL_0', 'score': 0.8726634979248047},
 {'label': 'LABEL_0', 'score': 0.8810698390007019},
 {'label': 'LABEL_0', 'score': 0.745476245880127},
 {'label': 'LABEL_0', 'score': 0.884869396686554},
 {'label': 'L